# devqubit UI Demo

This notebook populates a workspace with realistic quantum experiments, then launches the web UI for exploration.

## What We'll Create

We run several projects to demonstrate the UI's capabilities:

| Project | Description | UI Features Demonstrated |
|---------|-------------|--------------------------|
| `ghz_fidelity` | GHZ state fidelity across backends | Baseline comparison, run details |
| `shot_scaling` | Bell circuit with varying shots | Grouped runs (sweeps) |
| `qft_depth` | QFT scaling with qubit count | Metrics visualization |
| `transpilation` | Original vs optimized circuits | Run comparison (diff) |
| `failed_runs` | Simulated failures + recovery | Error states, filtering |

## Running This Notebook

1. Execute all cells to create the demo data
2. The final cell starts the web server
3. Open the printed URL in your browser
4. Interrupt the kernel (Ctrl+C) to stop the server

The workspace is created in `.devqubit_ui_demo/` and can be deleted after the demo.

### 1. Setup and Dependencies

First, let's check that all required packages are available and set up the workspace.

In [ ]:
from __future__ import annotations

from importlib.metadata import entry_points

import math
import shutil
import uuid
from pathlib import Path

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

from devqubit import track, Config
from devqubit.storage import create_store, create_registry


def installed_adapters() -> set[str]:
    eps = entry_points().select(group="devqubit.adapters")
    return {ep.name for ep in eps}


def installed_components() -> set[str]:
    eps = entry_points().select(group="devqubit.components")
    return {ep.name for ep in eps}


if [n for n in ("qiskit", "qiskit-runtime") if n not in installed_adapters()]:
    raise ImportError(
        "devqubit Qiskit adapter(s) missing: "
        "Install with: pip install 'devqubit[qiskit]'"
    )

if "ui" not in installed_components():
    raise ImportError("devqubit UI missing." "Install with: pip install 'devqubit[ui]'")
else:
    print("Qiskit adapters and UI extension available!")


# Check for fake backends (optional but recommended)
try:
    from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeAthensV2

    FAKE_BACKENDS_AVAILABLE = True
    print("Fake backends available (qiskit-ibm-runtime)")
except ImportError:
    FAKE_BACKENDS_AVAILABLE = False
    print("Fake backends not available (optional)")

### Initialize Workspace

devqubit stores experiment data in a **workspace** directory containing:
- **Registry** — Run metadata (parameters, metrics, tags)
- **Object Store** — Artifacts (circuits, results, device snapshots)

In [ ]:
# Create a fresh workspace for this demo
WORKSPACE = Path(".devqubit_ui_demo")

if WORKSPACE.exists():
    shutil.rmtree(WORKSPACE)
WORKSPACE.mkdir(parents=True)

# Initialize devqubit components
config = Config(root_dir=WORKSPACE)
registry = create_registry(config=config)
store = create_store(config=config)

print(f"Workspace: {WORKSPACE.resolve()}")

### Get Available Backends

We'll use a mix of ideal (AerSimulator) and noisy (fake backends) simulators to demonstrate how devqubit tracks experiments across different execution environments.

In [ ]:
def get_available_backends():
    """Get available backends for simulation."""
    backends = {}

    if FAKE_BACKENDS_AVAILABLE:
        try:
            backends["fake_manila"] = FakeManilaV2()
        except Exception:
            pass
        try:
            backends["fake_athens"] = FakeAthensV2()
        except Exception:
            pass

    return backends


backends = get_available_backends()
print(f"Available fake backends: {list(backends.keys()) or 'None'}")
print("AerSimulator: Always available (ideal simulation)")

### 2. Circuit Builders

Let's define some common quantum circuits we'll use throughout the demo.

In [ ]:
def create_ghz_circuit(n_qubits: int) -> QuantumCircuit:
    """
    Create a GHZ state preparation circuit.

    The GHZ state is a maximally entangled state:
    |GHZ⟩ = (|00...0⟩ + |11...1⟩) / √2

    It's highly sensitive to noise, making it ideal for
    benchmarking quantum hardware fidelity.
    """
    qc = QuantumCircuit(n_qubits, n_qubits, name=f"ghz_{n_qubits}")
    qc.h(0)  # Create superposition on first qubit
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)  # Propagate entanglement
    qc.measure(range(n_qubits), range(n_qubits))
    return qc


def create_bell_circuit() -> QuantumCircuit:
    """
    Create a Bell state circuit (2-qubit GHZ).

    |Φ+⟩ = (|00⟩ + |11⟩) / √2

    The simplest entangled state, perfect for quick tests.
    """
    qc = QuantumCircuit(2, 2, name="bell")
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    return qc


def create_qft_circuit(n_qubits: int) -> QuantumCircuit:
    """
    Create a Quantum Fourier Transform circuit.

    QFT is a key subroutine in many quantum algorithms
    (Shor's algorithm, phase estimation). Circuit depth
    grows as O(n²), making it useful for scaling studies.
    """
    qc = QuantumCircuit(n_qubits, n_qubits, name=f"qft_{n_qubits}")

    for i in range(n_qubits):
        qc.h(i)
        for j in range(i + 1, n_qubits):
            qc.cp(math.pi / (2 ** (j - i)), j, i)

    # Swap for standard QFT ordering
    for i in range(n_qubits // 2):
        qc.swap(i, n_qubits - i - 1)

    qc.measure(range(n_qubits), range(n_qubits))
    return qc

Let's visualize these circuits:

In [ ]:
print("GHZ Circuit (4 qubits):")
print(create_ghz_circuit(4).draw())
print("\nBell Circuit:")
print(create_bell_circuit().draw())
print("\nQFT Circuit (3 qubits):")
print(create_qft_circuit(3).draw())

### 3. Project: GHZ Fidelity Across Backends

Our first project compares GHZ state fidelity across different backends. This demonstrates:

- **Automatic artifact capture** — devqubit's `run.wrap()` automatically logs circuits, device snapshots, and results
- **Baseline setting** — Mark the ideal simulator as the reference for comparisons
- **Multi-backend tracking** — Same circuit, different execution environments

### Why GHZ for Benchmarking?

The GHZ state should produce only two outcomes: all zeros or all ones. Any other outcome indicates an error. This makes fidelity calculation simple:

$\text{Fidelity} = P(|00...0\rangle) + P(|11...1\rangle)$

In [ ]:
print("=" * 50)
print("Project: ghz_fidelity")
print("=" * 50)

# Build list of backends to test
backend_list = [("aer_simulator", AerSimulator())]  # Always include ideal
backend_list.extend(list(backends.items())[:2])  # Add up to 2 fake backends

baseline_id = None

for backend_name, backend in backend_list:
    n_qubits = min(5, getattr(backend, "num_qubits", 5))
    circuit = create_ghz_circuit(n_qubits)
    shots = 4096

    try:
        # The `track` context manager creates a new run
        with track(
            project="ghz_fidelity",
            run_name=f"GHZ_{backend_name}",
            config=config,
        ) as run:

            # Wrap the backend
            tracked = run.wrap(backend)

            # Log experiment parameters
            run.log_params(
                {
                    "n_qubits": n_qubits,
                    "shots": shots,
                    "backend": backend_name,
                }
            )

            # Execute the circuit
            job = tracked.run(circuit, shots=shots)
            counts = job.result().get_counts()

            # Calculate fidelity
            zeros = "0" * n_qubits
            ones = "1" * n_qubits
            fidelity = (counts.get(zeros, 0) + counts.get(ones, 0)) / shots

            # Log metrics
            run.log_metrics(
                {
                    "fidelity": fidelity,
                    "p_zeros": counts.get(zeros, 0) / shots,
                    "p_ones": counts.get(ones, 0) / shots,
                }
            )

            # Set ideal simulator as baseline
            if backend_name == "aer_simulator":
                baseline_id = run.run_id
                run.set_tag("role", "baseline")

            print(f"  {backend_name}: fidelity={fidelity:.4f}")

    except Exception as e:
        print(f"  {backend_name}: FAILED - {e}")

# Register the baseline for this project
if baseline_id:
    registry.set_baseline("ghz_fidelity", baseline_id)
    print(f"\n✓ Baseline set: {baseline_id[:12]}...")

### 4. Project: Shot Scaling Study

This project demonstrates **grouped runs** — a sweep where all runs share a `group_id`. This is useful for:

- Hyperparameter searches
- Scaling studies
- Ablation experiments

### Statistical Error and Shot Count

The standard error of a probability estimate scales as:

$\sigma = \sqrt{\frac{p(1-p)}{n}}$

where $p$ is the measured probability and $n$ is the shot count. More shots = lower error.

In [ ]:
print("=" * 50)
print("Project: shot_scaling")
print("=" * 50)

# Use fake backend if available, otherwise AerSimulator
backend = backends.get("fake_manila", AerSimulator())
backend_name = "fake_manila" if "fake_manila" in backends else "aer_simulator"

# Create a unique group ID for this sweep
sweep_id = f"shots_{uuid.uuid4().hex[:8]}"
circuit = create_bell_circuit()

print(f"Backend: {backend_name}")
print(f"Group ID: {sweep_id}")
print()

for shots in [256, 1024, 4096, 8192]:
    try:
        with track(
            project="shot_scaling",
            run_name=f"bell_{shots}_shots",
            config=config,
            group_id=sweep_id,  # Links this run to the sweep
            group_name="Shot Count Sweep",  # Human-readable name
        ) as run:
            tracked = run.wrap(backend)

            run.log_params(
                {
                    "shots": shots,
                    "backend": backend_name,
                }
            )

            job = tracked.run(circuit, shots=shots)
            counts = job.result().get_counts()

            fidelity = (counts.get("00", 0) + counts.get("11", 0)) / shots
            std_err = (fidelity * (1 - fidelity) / shots) ** 0.5

            run.log_metrics(
                {
                    "fidelity": fidelity,
                    "std_error": std_err,
                }
            )

            print(f"  shots={shots:5d}: fidelity={fidelity:.4f} ± {std_err:.4f}")

    except Exception as e:
        print(f"  shots={shots}: FAILED - {e}")

### 5. Project: QFT Depth Scaling

This project studies how circuit depth affects execution. QFT depth grows quadratically with qubit count, making it a good test for scaling behavior.

### Why Depth Matters

- **Noisy devices**: Error accumulates with each gate layer
- **Coherence time**: Deeper circuits may exceed T1/T2 limits
- **Compiler optimization**: Transpiler may reduce depth significantly

In [ ]:
print("=" * 50)
print("Project: qft_depth")
print("=" * 50)

sweep_id = f"qft_{uuid.uuid4().hex[:8]}"

for n_qubits in [2, 3, 4, 5]:
    try:
        circuit = create_qft_circuit(n_qubits)

        with track(
            project="qft_depth",
            run_name=f"qft_{n_qubits}q",
            config=config,
            group_id=sweep_id,
            group_name="QFT Scaling",
        ) as run:
            tracked = run.wrap(backend)

            run.log_params(
                {
                    "n_qubits": n_qubits,
                    "depth": circuit.depth(),
                    "backend": backend_name,
                }
            )

            job = tracked.run(circuit, shots=4096)
            counts = job.result().get_counts()

            run.log_metrics(
                {
                    "unique_outcomes": len(counts),
                    "max_prob": max(counts.values()) / 4096,
                }
            )

            run.set_tag("algorithm", "QFT")

            print(
                f"  n={n_qubits}: depth={circuit.depth():2d}, unique_outcomes={len(counts)}"
            )

    except Exception as e:
        print(f"  n={n_qubits}: FAILED - {e}")

### 6. Project: Transpilation Comparison

This project compares the same logical circuit with different optimization levels.

Qiskit Runtime V2 primitives require ISA-compatible circuits.
devqubit handles this automatically with three transpilation modes:
- 'auto': Transpile only if circuits aren't ISA-compatible (default)
- 'managed': Always transpile through devqubit
- 'manual': User handles transpilation (devqubit passes circuits as-is)

This project demonstrates how devqubit tracks transpilation metadata,
letting you see exactly what happened to your circuits.

The transpilation optimizes circuits for specific hardware, potentially:

- Reducing gate count and depth
- Mapping to native gate set
- Optimizing qubit routing

We will track all three variants to see if optimization improves fidelity.

In [ ]:
if backends:
    backend = list(backends.values())[0]
    backend_name = list(backends.keys())[0]

    # High-level circuit (NOT ISA-compatible)
    circuit = create_ghz_circuit(4)

    print(f"Backend: {backend_name}")
    print(f"Original circuit: {circuit.num_qubits} qubits, depth {circuit.depth()}")
    print()

    # Mode 1: Auto (default) - devqubit transpiles if needed
    # This is the recommended mode for most users
    try:
        with track(
            project="transpilation",
            run_name="transpilation_auto",
            config=config,
        ) as run:
            tracked = run.wrap(backend)

            run.log_params(
                {
                    "backend": backend_name,
                    "transpilation_mode": "auto",
                    "original_depth": circuit.depth(),
                }
            )

            # Pass high-level circuit - devqubit handles ISA compatibility
            job = tracked.run(circuit, shots=4096)
            counts = job.result().get_counts()

            fidelity = (counts.get("0000", 0) + counts.get("1111", 0)) / 4096
            run.log_metrics({"fidelity": fidelity})
            run.set_tag("mode", "auto")

            print(f"  auto:    fidelity={fidelity:.4f} (devqubit transpiles if needed)")

    except Exception as e:
        print(f"  auto: FAILED - {e}")

    # Mode 2: Managed - devqubit always transpiles with custom options
    # Use this when you want devqubit to handle transpilation with specific settings
    try:
        with track(
            project="transpilation",
            run_name="transpilation_managed",
            config=config,
        ) as run:
            tracked = run.wrap(backend)

            run.log_params(
                {
                    "backend": backend_name,
                    "transpilation_mode": "managed",
                    "optimization_level": 3,
                    "original_depth": circuit.depth(),
                }
            )

            # devqubit transpiles with optimization_level=3
            job = tracked.run(
                circuit,
                shots=4096,
                devqubit_transpilation_mode="managed",
                devqubit_transpilation_options={"optimization_level": 3},
            )
            counts = job.result().get_counts()

            fidelity = (counts.get("0000", 0) + counts.get("1111", 0)) / 4096
            run.log_metrics({"fidelity": fidelity})
            run.set_tag("mode", "managed")

            print(
                f"  managed: fidelity={fidelity:.4f} (devqubit transpiles with opt=3)"
            )

    except Exception as e:
        print(f"  managed: FAILED - {e}")

    # Mode 3: Manual - User pre-transpiles, devqubit passes through
    # Use this when you need full control over transpilation
    try:
        transpiled = transpile(circuit, backend=backend, optimization_level=3)

        with track(
            project="transpilation",
            run_name="transpilation_manual",
            config=config,
        ) as run:
            tracked = run.wrap(backend)

            run.log_params(
                {
                    "backend": backend_name,
                    "transpilation_mode": "manual",
                    "optimization_level": 3,
                    "original_depth": circuit.depth(),
                    "transpiled_depth": transpiled.depth(),
                }
            )

            # Pass pre-transpiled circuit - devqubit logs it as-is
            job = tracked.run(
                transpiled,
                shots=4096,
                devqubit_transpilation_mode="manual",
            )
            counts = job.result().get_counts()

            fidelity = (counts.get("0000", 0) + counts.get("1111", 0)) / 4096
            run.log_metrics({"fidelity": fidelity})
            run.set_tag("mode", "manual")

            print(
                f"  manual:  fidelity={fidelity:.4f} (user transpiled, depth={transpiled.depth()})"
            )

    except Exception as e:
        print(f"  manual: FAILED - {e}")

    print()
    print("In the UI, check each run's 'execute' record to see transpilation metadata:")
    print("  - transpilation_mode: which mode was used")
    print("  - transpiled_by_devqubit: whether devqubit performed transpilation")
    print("  - transpilation_needed: whether circuits required transpilation")

else:
    print("  [SKIP] No fake backends available for transpilation demo")

### 7. Project: Failed Runs

Real experiments fail. devqubit tracks failed runs too, preserving:

- Parameters logged before failure
- Partial artifacts (if any)
- Error information

This is crucial for debugging and understanding failure patterns.

In [ ]:
print("=" * 50)
print("Project: failed_runs")
print("=" * 50)

# Create some failed runs
for i in range(2):
    try:
        with track(
            project="failed_runs",
            run_name=f"failing_attempt_{i + 1}",
            config=config,
        ) as run:
            run.log_params({"attempt": i + 1})

            # Simulate a failure
            raise RuntimeError("Simulated backend timeout")

    except RuntimeError:
        print(f"  Failed run #{i + 1} created")

# Create a successful run for comparison
with track(project="failed_runs", run_name="recovery_run", config=config) as run:
    tracked = run.wrap(AerSimulator())
    run.log_params({"attempt": 3, "recovered": True})

    job = tracked.run(create_bell_circuit(), shots=1000)
    job.result()

    run.log_metric("success", 1.0)
    print("  Successful run #3 created")

---
## 8. Summary

Let's see what we've created:

In [ ]:
all_runs = registry.list_runs(limit=100)
all_projects = registry.list_projects()
all_groups = registry.list_groups()

print("=" * 50)
print("WORKSPACE SUMMARY")
print("=" * 50)
print(f"  Total runs:   {len(all_runs)}")
print(f"  Projects:     {len(all_projects)}")
print(f"  Groups:       {len(all_groups)}")
print()
print("Projects:")
for project in all_projects:
    count = len([r for r in all_runs if r.get("project") == project])
    print(f"  - {project}: {count} runs")
print()
print("Groups (sweeps):")
for group in all_groups:
    print(f"  - {group.get('group_name', group.get('group_id', 'unnamed'))}")

---
## 9. Launch the Web UI

Now let's launch the devqubit web UI to explore our experiments visually!

The UI provides:
- **Runs list** — Browse all experiments with filtering
- **Run details** — View parameters, metrics, and artifacts
- **Comparison** — Diff two runs side-by-side
- **Groups** — Navigate experiment sweeps
- **Projects** — Overview of all projects

### ⚠️ Note
Running the cell below will start a web server. The cell will block until you stop the server (interrupt the kernel).

In [ ]:
from devqubit.ui import run_server

run_server(workspace=str(WORKSPACE))

---
## 10. Cleanup (Optional)

Run this cell to remove the demo workspace:

In [ ]:
# Uncomment to delete the workspace
shutil.rmtree(WORKSPACE)
print(f"Removed: {WORKSPACE}")